# 파일 업로드

[링크]에서 만든 csv 데이터 압축 파일을 업로드합니다.

왼쪽 파일 탭에서 업로드 하거나 아래의 코드를 실행합니다.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving cow_csv.zip to cow_csv.zip
User uploaded file "cow_csv.zip" with length 8685509 bytes


업로드한 파일의 압축을 해제합니다.

In [ ]:
!unzip -q cow_csv.zip # 압축파일 이름 확인

In [ ]:
!ls

cow  cow_csv.zip  sample_data


In [ ]:
!ls cow

'월별 lely 활동량 (24.01).csv'	'월별 lely 활동량 (24.07).csv'
'월별 lely 활동량 (24.02).csv'	'월별 lely 활동량 (24.08).csv'
'월별 lely 활동량 (24.03).csv'	'월별 lely 활동량 (24.09).csv'
'월별 lely 활동량 (24.04).csv'	'월별 lely 활동량 (24.10).csv'
'월별 lely 활동량 (24.05).csv'	'월별 lely 활동량 (24.11).csv'
'월별 lely 활동량 (24.06).csv'	'월별 lely 활동량 (24.12)~12.22.csv'


# 데이터셋

## 1. 데이터 전처리



In [ ]:
import glob
import pandas as pd
import numpy as np

In [ ]:
csv_list = glob.glob('./cow/*.csv')
csv_list.sort()

tmp = [] # 데이터를 임시저장할  빈 리스트
for csvfile in csv_list:
    try:
        print(csvfile)
        df = pd.read_csv(csvfile, low_memory=False)

        if '시간' in df.columns:
            df.rename(columns={'시간': '시간(시:분)'}, inplace=True)

        selected_cols = ['개체 번호', '날짜', '시간(시:분)', '활동량', '전체 반추 시간(분)', '발정 확률']
        df = df[selected_cols]

        df = df.dropna(axis=0, inplace=False)

        tmp.append(df)
    except Exception as e:
        print(f"{csvfile} 처리 중 오류 발생:", e)

data = pd.concat(tmp, ignore_index=True)

./cow/월별 lely 활동량 (24.01).csv
./cow/월별 lely 활동량 (24.02).csv
./cow/월별 lely 활동량 (24.03).csv
./cow/월별 lely 활동량 (24.04).csv
./cow/월별 lely 활동량 (24.05).csv
./cow/월별 lely 활동량 (24.06).csv
./cow/월별 lely 활동량 (24.07).csv
./cow/월별 lely 활동량 (24.08).csv
./cow/월별 lely 활동량 (24.09).csv
./cow/월별 lely 활동량 (24.10).csv
./cow/월별 lely 활동량 (24.11).csv
./cow/월별 lely 활동량 (24.12)~12.22.csv


In [ ]:
# 날짜와 시간 합쳐서 datetime
data['datetime'] = pd.to_datetime(data['날짜'] + ' ' + data['시간(시:분)'])
# 필요한 컬럼만 추출
data = data[['개체 번호', 'datetime', '활동량', '전체 반추 시간(분)', '발정 확률']]
data.sort_values(['개체 번호', 'datetime'], inplace=True)
data.head()

/tmp/ipython-input-9-2639366942.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.sort_values(['개체 번호', 'datetime'], inplace=True)


,개체 번호,datetime,활동량,전체 반추 시간(분),발정 확률
87,1,2024-01-01 00:00:00,53.0,465.0,-2.0
225,1,2024-01-01 02:00:00,37.0,483.0,-2.0
363,1,2024-01-01 04:00:00,34.0,469.0,-3.0
501,1,2024-01-01 06:00:00,45.0,486.0,-4.0
639,1,2024-01-01 08:00:00,47.0,468.0,-7.0


In [ ]:
print(data.shape) # (504678, 5)

(504678, 5)


정제한 데이터를 저장하고 싶다면,

In [ ]:
data.to_csv('cow_data.csv', encoding='utf-8-sig', index=False)

In [ ]:
files.download('cow_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

===============================================================


***TIP:***

세션 재시작 시 위 과정을 반복하지 말고 저장한 csv 파일만 간편하게 불러오세요!

In [ ]:
files.upload()

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('dairy_data.csv', low_memory=False)

===============================================================

## 2. 데이터셋 구성

todo:
- 개체별 train/val split
- 정규화
- https://velog.io/@choonsik_mom/pytorch%EB%A1%9C-LSTM-%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0 참고하여 다시 작성


### 데이터 정규화
* 정규화 없이 실험해보고, 정규화 적용 후 실험 결과와 비교를 추천합니다.
* Min-max 정규화를 적용합니다.

In [ ]:
def min_max_norm(col):
    return (col - col.min()) / (col.max() - col.min())

In [ ]:
data_norm = data.copy()
data_norm['발정 확률'] = min_max_norm(data_norm['발정 확률'])
data_norm.head()

,개체 번호,datetime,활동량,전체 반추 시간(분),발정 확률
87,1,2024-01-01 00:00:00,53.0,465.0,0.500000
225,1,2024-01-01 02:00:00,37.0,483.0,0.500000
363,1,2024-01-01 04:00:00,34.0,469.0,0.494898
501,1,2024-01-01 06:00:00,45.0,486.0,0.489796
639,1,2024-01-01 08:00:00,47.0,468.0,0.474490


### train/val/test split

train/val을 시기로 나눌 것인가? 개체별로 나눌 것인가?
(일반적으로는 시기)

데이터의 `'개체 번호'`를 확인합니다.

In [ ]:
data['개체 번호'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98,  99, 101, 102, 104, 105, 106, 107, 108,
       110, 111, 112, 113, 114, 116, 117, 118, 121, 122, 124, 125, 128,
       129, 131, 132, 133, 136, 137, 138, 139, 140, 143, 144, 145, 146,
       149, 150, 151, 153, 155, 156, 157, 158, 159, 161, 162, 163, 164,
       166, 167, 170, 199, 201, 203, 204, 205, 208, 211, 214, 215, 216,
       218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230,
       231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 615, 65

In [ ]:
len(data['개체 번호'].unique())

185

`'개체 번호'`의 끝 번호는 690이지만, `len(data['개체 번호'].unique())`는 185인 것을 확인했습니다.

총 개체 수는 '185' 입니다.

In [ ]:
num_idx = len(data['개체 번호'].unique())
idx_list = np.random.permutation(data['개체 번호'].unique())

# train/val
train_idx = idx_list[:int(num_idx*0.8)]
val_idx   = idx_list[int(num_idx*0.8):]

print(f"train 개체 수: {len(train_idx)}")
print(f"val 개체 수: {len(val_idx)}")

# # train/val/test
# train_idx = idx_list[:int(num_idx*0.8)]
# val_idx   = idx_list[int(num_idx*0.8):int(num_idx*0.9)]
# test_idx = idx_list[int(num_idx*0.9):]

# print(f"train 개체 수: {len(train_idx)}")
# print(f"val 개체 수: {len(val_idx)}")
# print(f"test 개체 수: {len(test_idx)}")

train 개체 수: 148
val 개체 수: 37


In [ ]:
print(train_idx)
print(val_idx)
# print(test_idx)

[ 25  31  97 208  87 157 132  14 219 221  84 215  66  19 222 229 128  27
  69 162  15  65  26 164  44  40 166  45 199 170 138  83  56 167 224  73
  96 218  17  35   8  42  18  62 214 149  61  52  92 204 116 659   2  58
  11 118  34 690  22 660 153 236 112 227  90 161  68  37   9  94  72  67
  41   7 122  21 661  28  99  70 163 121  91 145 159 129  30 108 104 223
 144  60  13 158  29 102  39  76  93  63  16 140   4  85 203  86 239   1
  33 230  10 125 216 235 232  59  80 106  71 137 136  75 143  82  81  88
 231 105 101 228 111 107  47  98 240  54 211 110 139 662  55  23  57 201
  89 151  50 205]
[ 64 146 113  24   5  43   6 156 238 615   3  77  12  53 233 155  48 234
  51  20 124 131 225  36  46 133  38  32 150 226 114  95  79 237  78 117
 220]


## 2. 시퀀스 정의

In [ ]:
def make_sequences(df, seq_len=4):
    sequences = []
    features = ['활동량', '전체 반추 시간(분)']
    for cow_id, group in df.groupby('개체 번호'):
        vals = group[features + ['발정 확률']].values
        for i in range(len(vals) - seq_len):
            seq_x = vals[i:i+seq_len, :2]         # (seq_len, 2)
            seq_y = vals[i+seq_len, 2]            # scalar
            sequences.append((seq_x, seq_y))
    return sequences

seq_len = 12 # 데이터 특징과 모델 성능을 고려하여 적절히 조정
sequences = make_sequences(data, seq_len)
print(f"총 시퀀스 수: {len(sequences)}")

## 3. PyTorch 데이터셋 정의

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

데이터셋 클래스 생성&정의

In [ ]:
class CowEstrusDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):
        x, y = self.sequences[idx]
        return torch.FloatTensor(x), torch.FloatTensor([y])


데이터 로더

In [ ]:
# train/val split
split = int(len(sequences) * 0.8)
train_ds = CowEstrusDataset(sequences[:split])
val_ds   = CowEstrusDataset(sequences[split:])

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=64)

# LSTM 회귀 모델

모델 정의

In [ ]:
import torch.nn as nn

In [ ]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, sequence_length):


In [ ]:
# class LSTM(nn.Module):
#     def __init__(self, input_size=2, hidden_size=32, num_layers=2):
#         super().__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
#                             batch_first=True)
#         self.fc   = nn.Linear(hidden_size, 1)
#     def forward(self, x):
#         # x: (batch, seq_len, input_size)
#         out, _ = self.lstm(x)
#         last   = out[:, -1, :]          # 마지막 타임스텝
#         return self.fc(last)            # (batch, 1)

# 학습 및 평가

In [ ]:
device   = 'cuda' if torch.cuda.is_available() else 'cpu'
model    = LSTM().to(device)

In [ ]:
criterion= nn.MSELoss()
optimizer= torch.optim.Adam(model.parameters(), lr=0.005)

In [ ]:
def train_epoch():
    model.train()
    total = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        total += loss.item()
    return total / len(train_loader)

In [ ]:
def eval_epoch():
    model.eval()
    total = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            total += criterion(pred, y).item()
    return total / len(val_loader)

In [ ]:
epochs = 20
for ep in range(1, epochs+1):
    tr = train_epoch()
    va = eval_epoch()
    print(f"Epoch {ep:02d} ▶ train_loss={tr:.4f}, val_loss={va:.4f}")

# 시각화

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 검증셋에 대해 예측 & 판정
model.eval()
preds, trues = [], []
with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        p = model(x).cpu().squeeze().numpy()
        preds.extend(p)
        trues.extend(y.squeeze().numpy())

# 회귀 예측 곡선
plt.figure(figsize=(8,4))
plt.plot(trues, label='True')
plt.plot(preds, label='Pred')
plt.legend()
plt.title("Estrus Probability: True vs Pred")
plt.show()

# 25 이상이면 발정(True), 미만이면 비발정(False)
pred_labels = np.array(preds) >= 25
true_labels = np.array(trues) >= 25

from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_labels, target_names=['No Estrus','Estrus']))
